---
# **1. Загрузка данных с MOEX и подготовка**

На этом этапе загружаются исторические данные о ценах акций с Московской биржи (MOEX).
Основная задача — собрать последовательность дневных цен закрытия по выбранным инструментам.

**Ключевые шаги:**

1. **Кэширование**: данные сохраняются в .parquet для оптимизации скорости
2. **Пагинация API**: MOEX API возвращает данные блоками, используется цикл для получения всех данных
3. **Синхронизация дат**: берётся пересечение торговых дней всех активов

Входные данные:
- $P_i(t)$ — цена закрытия актива $i$ в день $t$
- $T$ — количество торговых дней
- $n$ — количество активов

Выходные данные:
- $\mathbf{P} \in \mathbb{R}^{T \times n}$ — матрица цен закрытия

Синхронизация дат:
$$\mathbf{D}_{\text{final}} = \bigcap_{i=1}^{n} \mathbf{D}_i$$

где $\mathbf{D}_i$ — множество торговых дней актива $i$.

- **TICKERS**: список тикеров
- **START_DATE / END_DATE**: диапазон данных
- **CACHE_DIR**: директория для сохранения кэша
- **FORCE_RELOAD**: очистить кэш перед загрузкой

---

In [ ]:
!pip install arch
import numpy as np
import pandas as pd
import requests
import os
import time
from datetime import datetime
from pathlib import Path
import shutil
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from arch import arch_model
from typing import Dict, List, Tuple, Optional
import warnings

warnings.filterwarnings('ignore')

TICKERS = ['AFKS', 'AFLT', 'ALRS', 'BSPB', 'CBOM', 'CHMF', 'CNRU', 'ENPG', 'FLOT', 'GAZP', 'GMKN', 'HEAD', 'IRAO', 'LKOH', 'MAGN', 'MDMG', 'MOEX', 'MSNG', 'MTSS', 'NLMK', 'NVTK', 'OZON', 'PHOR', 'PIKK', 'PLZL', 'POSI', 'RENI', 'ROSN', 'RTKM', 'RUAL', 'SBER', 'SBERP', 'SNGS', 'SNGSP', 'SVCB', 'T', 'TATN', 'TATNP', 'TRNFP', 'UGLD', 'VKCO', 'VTBR', 'X5', 'YDEX']
START_DATE = '2024-06-01'
END_DATE = '2025-12-01'

CACHE_DIR = Path('./data_cache')
FORCE_RELOAD = True

if FORCE_RELOAD and CACHE_DIR.exists():
    shutil.rmtree(CACHE_DIR)

CACHE_DIR.mkdir(exist_ok=True)

def get_cache_path(ticker: str) -> Path:
    return CACHE_DIR / f"{ticker}_{START_DATE}_{END_DATE}.parquet"


def load_from_cache(ticker: str) -> pd.DataFrame:
    cache_path = get_cache_path(ticker)

    if cache_path.exists():
        try:
            df = pd.read_parquet(cache_path)
            print(f"   {ticker}: кэш ({len(df)} дней)")
            return df
        except:
            return None
    return None


def save_to_cache(ticker: str, df: pd.DataFrame) -> None:
    try:
        df.to_parquet(get_cache_path(ticker))
    except:
        pass


def fetch_from_moex_complete(ticker: str, start_date: str, end_date: str) -> pd.DataFrame:

    url = (
        f"https://iss.moex.com/iss/history/engines/stock/markets/shares/"
        f"boards/TQBR/securities/{ticker}.json"
    )

    all_rows = []
    start_idx = 0
    columns = None

    print(f"   ↓ {ticker}...", end=" ", flush=True)

    while True:
        params = {
            'from': start_date,
            'till': end_date,
            'sort_order': 'asc',
            'start': start_idx
        }

        try:
            response = requests.get(url, params=params, timeout=15)
            response.raise_for_status()
            data = response.json()

            if 'history' not in data or 'data' not in data['history']:
                break

            rows = data['history']['data']

            if columns is None:
                columns = data['history']['columns']

            if len(rows) == 0:
                break

            all_rows.extend(rows)
            start_idx += len(rows)

            print(".", end="", flush=True)

        except Exception as e:
            print(f"ERROR: {e}")
            return None

    if len(all_rows) == 0:
        print(f"No data")
        return None

    print(f" ({len(all_rows)} rows)")

    try:
        df = pd.DataFrame(all_rows, columns=columns)
        df = df[['TRADEDATE', 'CLOSE']].copy()
        df.columns = ['Date', 'Close']

        df['Date'] = pd.to_datetime(df['Date'])
        df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

        df = df.drop_duplicates(subset=['Date'])
        df = df.dropna()
        df = df.sort_values('Date').reset_index(drop=True)

        return df
    except Exception as e:
        print(f"Parse error: {e}")
        return None


def load_ticker_data(ticker: str) -> pd.DataFrame:
    print(f"{ticker}:", end="")

    df = load_from_cache(ticker)

    if df is None:
        df = fetch_from_moex_complete(ticker, START_DATE, END_DATE)

        if df is None:
            print(f" ERROR")
            return None

        save_to_cache(ticker, df)

    return df

print(f"Период: {START_DATE} → {END_DATE}")
print(f"Активов: {len(TICKERS)}\n")

all_data = {}

for ticker in TICKERS:
    df = load_ticker_data(ticker)
    if df is not None:
        all_data[ticker] = df

print(f"\n Loaded: {len(all_data)}/{len(TICKERS)}")

if len(all_data) == 0:
    raise ValueError(" Error: no data!")

for ticker in all_data:
    df = all_data[ticker]
    print(f"   {ticker:6s}: {df['Date'].min().date()} → {df['Date'].max().date()} ({len(df):3d} дней)")

valid_tickers = list(all_data.keys())
all_dates = set(all_data[valid_tickers[0]]['Date'])

for ticker in valid_tickers[1:]:
    all_dates = all_dates.intersection(set(all_data[ticker]['Date']))

print(f"Общих дней: {len(all_dates)}")

for ticker in all_data:
    df = all_data[ticker]
    df = df[df['Date'].isin(all_dates)].sort_values('Date').reset_index(drop=True)
    all_data[ticker] = df

first_ticker = valid_tickers[0]
priced_df = all_data[first_ticker][['Date', 'Close']].copy()
priced_df.columns = ['Date', first_ticker]

for ticker in valid_tickers[1:]:
    temp = all_data[ticker][['Date', 'Close']].copy()
    temp.columns = ['Date', ticker]
    priced_df = priced_df.merge(temp, on='Date', how='inner')

priced_df = priced_df.set_index('Date').sort_index()

print(f" Размер: {priced_df.shape[0]} дней × {priced_df.shape[1]} активов")
print(f"   Период: {priced_df.index[0].date()} → {priced_df.index[-1].date()}")

asset_names = list(priced_df.columns)
n_assets = len(asset_names)
T = len(priced_df)

print(f"Переменные:")
print(f"   priced_df   : DataFrame цен ({priced_df.shape})")
print(f"   asset_names : {asset_names}")
print(f"   n_assets    : {n_assets}")
print(f"   T           : {T} дней")